<a href="https://colab.research.google.com/github/telsayed/IR-in-Arabic/blob/master/Summer2021/notebooks/day1/IR_in_Arabic_Lab1_BooleanRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **IR in Arabic** - Summer 2021 lab day1


This is one of a series of Colab notebooks created for the **IR in Arabic** course. It demonstrates how can we build a Term-Document incidence matrix and a boolean retrieval model.

The **learning outcomes** of the this notebook are:


*   What is Colab and how can we run our code?
*   Build a Term-Document incidence Matrix and visualize it.
*   Build a boolean retrieval model that can answer boolean queries.



# What is colab?

**[Colaboratory](https://research.google.com/colaboratory/faq.html)**, or “Colab” for short, is a product from Google Research. Colab allows anybody to write and execute arbitrary python code through the browser, and is especially well suited to machine learning, data analysis and education. More technically, Colab is a hosted Jupyter notebook service that requires no setup to use, while providing free access to computing resources including GPUs.

# How can you have your own copy of this lab?

You have to make a copy of this notebook by clicking [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](), then go to **File->Save a copy in Drive**. This will create your version of this lab saved in your Google Drive.

### **Running a simple code on colab.**

In [ ]:
print("مرحبا بكم في دورة استرجاع المعلومات")

### **Build a Term-Document incidence Matrix and visualize it.**

Let's create a set of textual documents.

In [ ]:
 d1 = "هذا هو اليوم الأول من دورة استرجاع المعلومات"
 d2 = "الدورة باللغة العربية للطلاب العرب"
 d3 = "اليوم هو 30 مايو 2021"
 d4 = "نأمل أن تفيد هذه الدورة الطلاب العرب"
 d5 = "هل أنتم سعداء بهذه التجربة"
 data = {"doc1": d1,
         "doc2": d2, 
         "doc3": d3, 
         "doc4": d4,
         "doc5": d5}
data

Let's define our boolean operators.

In [ ]:
boolean_operators = {'AND', 'OR', 'NOT'}

We need to implement some helper functions below

In [ ]:
  #list of terms
  def get_terms(data):
        terms=[]
        for doc in data:
            for term in data[doc].split() :
                terms.append(term)
        return terms

   #list of unique terms
  def get_unique_terms(terms):
        unique_terms=[]
        for d in terms :
            if d not in unique_terms:
               unique_terms.append(d)
        return unique_terms

  #document collection terms
  def get_document_collection_terms(data):
        docs_colllection={}
        for doc in data:
            if doc not in boolean_operators :
                docs_colllection[doc]=get_unique_terms(data[doc].split())
        return docs_colllection
           
  def display_dict(dic):
        print("\n")
        for i in dic:
            print (i , " : " ,dic[i])
        print("\n")

We need to define some local variables.

In [ ]:
 ## list of terms from the data file collection
 terms=[]
 #list of unique terms
 unique_terms=[]
 #Document collection terms
 docs_terms_dict={}
 #TermDocumentIncidenceMatrix 
 term_docs_matrix={}

Let's check the output of our helper functions.

In [ ]:
print("### documents content ###")
display_dict(data)
terms= get_terms(data)
print ("\n### Terms in 5 docs ###\n" , *terms ,sep= " | ")
#unique_terms
unique_terms = get_unique_terms(terms)
print ("\n### Unique Terms in 5 DOCs###\n", *unique_terms ,sep=" | ")
#docs_terms
docs_terms_dict= get_document_collection_terms(data)
print ("\n###Document terms Collection ###" )
display_dict(docs_terms_dict)

Next, we will implement a fucntion to build a term-incidence matrix

In [ ]:
def term_document_incidence_matrix(docs_terms ,unique_terms):

        term_docs_matrix={}
        for term in unique_terms:
            vector=[]
            for c in docs_terms:

                if term in docs_terms[c]:
                    vector.append(1)
                else :
                    vector.append(0)

            term_docs_matrix[term]=vector
        return term_docs_matrix

    
def term_incidence_vector(term):
    return term_docs_matrix[term]

Let's build the term-document incidence matrix

In [ ]:
#build the term-document incidence matrix
term_docs_matrix=term_document_incidence_matrix(docs_terms_dict,unique_terms)
print("Term-Document incidence Matrix\n")
#formatted Display 
display_dict(term_docs_matrix)


Let's check the term incidence vector for the word العرب

In [ ]:
print("Incidence Vector of 'العرب' ",term_incidence_vector('العرب'))

Incidence Vector of 'العرب'  [0, 1, 0, 1, 0]


### **Buidling a Boolean retrieval model**

First, we need to implement our that tokenizes the query to differentiate between the terms and the boolean operators and select only those defined in our booleanModel class previously.

In [ ]:
#Query Filteration
#input : Query 
#output : List of terms of a given query which match with the terms in our datafiles
def query_filteration(query):
    qterms=[]
    splitted_query=query.split()
    for qterm in splitted_query:
        if qterm in unique_terms or qterm in boolean_operators:
          qterms.append(qterm)
    return qterms

Let's test our Query filteration function. 

In [ ]:
query="استرجاع AND المعلومات AND دورة"
qterms=query_filteration(query)
print(qterms)

In [ ]:
# Boolean Operator Processing 
# input : Boolean Operator ,Next term Incedence Vector ,Previous term Incedence Vector 
def boolean_operator_processing(bool_operator,prevV,nextV):
    if bool_operator == "AND":
      return [a & b for a, b in zip(prevV, nextV)]
    elif bool_operator=="OR" :
      return [a | b for a, b in zip(prevV, nextV)]
    elif bool_operator == "NOT":
      return [1-a for a in prevV]

Let's test our BooleanOperatorProcessing function

In [ ]:
v1=[0,1,0,1]
v2=[0,0,1,1]
v3=[]
print(boolean_operator_processing("AND",v1,v2))
print(boolean_operator_processing("OR",v1,v2))
print(boolean_operator_processing("NOT",v1,v3))

[0, 0, 0, 1]
[0, 1, 1, 1]
[1, 0, 1, 0]


In [ ]:
# Boolean retrieval fuction
# input : Query 
# output : if Document is relevant > 1 else >0
def boolean_retrieval(query):
        bitwiseop=""
        qterms=query_filteration(query)
        result=[]
        result_set={}
        has_previous_term=False
        has_not_operation=False
        inc_vec_prev=[]
        inc_vec_next=[]
        for term in qterms :
            if term not in boolean_operators:
                if  has_not_operation:
                    if has_previous_term:
                         inc_vec_next=boolean_operator_processing("NOT",term_incidence_vector(term),inc_vec_next)
                    else :
                        inc_vec_prev=boolean_operator_processing("NOT",term_incidence_vector(term),inc_vec_next)
                        result=inc_vec_prev
                    has_not_operation=False
                    
                elif  has_previous_term:
                    
                    inc_vec_next=term_incidence_vector(term)
                else :
                    
                    inc_vec_prev=term_incidence_vector(term)
                    result= inc_vec_prev
                    has_previous_term=True    
            
            elif term =="NOT":
                 has_not_operation=True
            
            else :
                bitwiseop=term
                
            if  len(inc_vec_next)!= 0  :
                result = boolean_operator_processing(bitwiseop,inc_vec_prev,inc_vec_next)
                inc_vec_prev=result
                has_previous_term=True
                inc_vec_next= []
    
        for i,doc in zip(result,data):
            result_set[doc]=i
        return result_set

Let's use our boolean retrieval model 

In [ ]:
query1 = "العرب AND NOT المعلومات"
query2 = "استرجاع AND المعلومات AND NOT العربية"
query3 = "استرجاع OR الدورة OR سعداء"
print("query1 boolean retrieval result ",boolean_retrieval(query1))
print("query2 boolean retrieval ",boolean_retrieval(query2))
print("query3 boolean retrieval ",boolean_retrieval(query3))

### **Exercise1**

Find the documents that contains the word **سعداء** or **الدورة** but does not contain 
the word **العرب**

### **Exercise2**
Using the sentences given below create your documents, build your term-document incidence matrix, create three different queries similar to the ones introduced during the lab, and retrieve the results of your queries.


1. قضينا في الخليل ستة أشهر ثم بدأ أهل البلد يتوزعون منهم من أراد اللحاق بأقرباء له في طولكرم أو نابلس أو جنين و منهم من تسلل عائدا الى الجليل و منهم من ذهب الى سوريا
2.   سيذهلني أن مريم و هي تسترجع ما جرى و تنقله الى أخيها تذكر تفاصيل الحوار 
بالنقطة و الفاصلة
3. نعم زرت بيروت زرتها بعد خمس سنين من الغياب
4.وعدتك يا حسن أن أتم هذا الكتاب و لكنني حين وصلت الى هذا الجزء من الحكاية   تيقنت أنني لن أستطيع
5. حكت لي ممرضة من سكان شاتيلا أنها ذهبت الى مستشفى عكا يوم السبت بعد رحيل 
القوات من المنطقة

**ملحوظة:** الجمل مقتبسة من رواية الطنطورية لرضوى عاشور


### **References**


*   [Information retrieval models.](https://github.com/YoucefBYu/Information-Retrieval-Models)

